```

                             Alexey Kuleshevich
                             
                     Monadic Warsaw - 14th November 2023
                        
                 Type safety and inheritance in Haskell with
                   phantom types, type families and lenses.
                             
                         Cardano Ledger Case Study
                             
```

## What is a phantom type?

### Vessel that carries type information

> A __phantom type__ is a parameterised type whose parameters do not all appear 
on the right-hand side of its definition.

```haskell
data Proxy a = Proxy
  deriving Show
```

## Where can we use phantom types?

### To resolve type ambiguities

```haskell
printMempty :: IO ()
printMempty = print mempty
```

Will result in:

```
    • Ambiguous type variable ‘a0’ arising from a use of ‘print’
      prevents the constraint ‘(Show a0)’ from being solved.
      Probable fix: use a type annotation to specify what ‘a0’ should be.
      ...
    • Ambiguous type variable ‘a0’ arising from a use of ‘mempty’
      prevents the constraint ‘(Monoid a0)’ from being solved.
      Probable fix: use a type annotation to specify what ‘a0’ should be.
```

In [1]:
{-# LANGUAGE ScopedTypeVariables #-}
import Data.Proxy

printMempty :: forall m. (Monoid m, Show m) => Proxy m -> IO ()
printMempty _ = print (mempty :: m)

printMempty (Proxy :: Proxy (Maybe ()))

Nothing

## Where else can we use phantom types?

### To improve type safety

In [2]:
import Data.ByteString (ByteString)

ffiHashBlake2b224 :: ByteString -> ByteString
ffiHashBlake2b224 = error "FFI binding to Libsodium"

ffiHashBlake2b256 :: ByteString -> ByteString
ffiHashBlake2b256 = error "FFI binding to Libsodium"

```haskell
newtype HashBlake2b224 = HashBlake2b224 ByteString
newtype HashBlake2b256 = HashBlake2b256 ByteString

hashBlake2b224 :: ByteString -> HashBlake2b224
hashBlake2b224 = HashBlake2b224 . ffiHashBlake2b224

hashBlake2b256 :: ByteString -> HashBlake2b256
hashBlake2b256 = HashBlake2b256 . ffiHashBlake2b256
```

In [3]:
newtype Hash h = Hash ByteString
  deriving (Eq, Ord, Show)

class HashAlgorithm h where
  hash :: ByteString -> Hash h

In [4]:
data Blake2b224

instance HashAlgorithm Blake2b224 where
  hash = Hash . ffiHashBlake2b224

data Blake2b256

instance HashAlgorithm Blake2b256 where
  hash = Hash . ffiHashBlake2b256

In [5]:
:set -XOverloadedStrings

let h28 = hash "Cardano" :: Hash Blake2b224

:t h28

h28 :: Hash Blake2b224

In [6]:
import Data.Coerce (coerce)

h32 = coerce h28 :: Hash Blake2b256

:t h32

h32 :: Hash Blake2b256

Protect with:
* If a `newtype` is used, then do not export the constructor
* And change the role from `phantom` to `nominal`

```haskell
{-# LANGUAGE RoleAnnotations #-}
module Hash (Hash, HashAlgorithm (..), Blake2b224, Blake2b256) where

newtype Hash h = Hash ByteString
  deriving (Eq, Ord, Show)

type role Hash nominal
```

```haskell
h32 = coerce h28 :: Hash Blake2b256
```
```
    • Couldn't match type ‘Blake2b224’ with ‘Blake2b256’
        arising from a use of ‘coerce’
    • In the expression: coerce h28 :: Hash Blake2b256
      In an equation for ‘h32’: h32 = coerce h28 :: Hash Blake2b256
```

## More sophisticated phantom type uses

```haskell
newtype KeyHash = KeyHash (Hash Blake2b224)
  deriving (Eq, Ord, Show)

delegate
  :: KeyHash -- ^ Delegation KeyHash
  -> KeyHash -- ^ StakePool KeyHash
  -> Map KeyHash KeyHash
  -> Map KeyHash KeyHash
delegate staking pool = Map.insert staking pool
```

```haskell
delegate pool staking = Map.insert pool staking
```

### Close phantom types with DataKinds

In [ ]:
:set -XDataKinds -XKindSignatures

data KeyRole
  = Payment
  | Staking
  | StakePool
  | Minting

newtype KeyHash (r :: KeyRole) = KeyHash (Hash Blake2b224)
  deriving (Eq, Ord, Show)

In [ ]:
import qualified Data.Map as Map

delegate
  :: KeyHash 'Staking
  -> KeyHash 'StakePool
  -> Map.Map (KeyHash 'Staking) (KeyHash 'StakePool)
  -> Map.Map (KeyHash 'Staking) (KeyHash 'StakePool)
delegate = Map.insert

In [ ]:
delegate
  :: KeyHash 'Staking
  -> KeyHash 'StakePool
  -> Map.Map (KeyHash 'Staking) (KeyHash 'StakePool)
  -> Map.Map (KeyHash 'Staking) (KeyHash 'StakePool)
delegate staking pool = Map.insert pool staking

: 

## Lens

With [`microlens`](https://hackage.haskell.org/package/microlens) package.

In [10]:
import Lens.Micro

:t lens

lens :: forall s a b t. (s -> a) -> (s -> b -> t) -> Lens s t a b

```haskell
data TxOut = TxOut
  { txOutAddress :: Address
  , txOutValue :: Coin
  }

txOutAddressL :: Lens' TxOut Address
txOutAddressL = 
  lens (\txOut -> txOutAddress txOut)
       (\txOut address -> txOut {txOutAddress = address})
```

In [11]:
:t (&) -- From Data.Function, same as `flip ($)`
:t (.~)
:t (^.)

(&) :: forall a b. a -> (a -> b) -> b

(.~) :: forall s t a b. ASetter s t a b -> b -> s -> t

(^.) :: forall s a. s -> Getting a s a -> a

```haskell
>>> let txOut' = txOut & txOutAddressL .~ "0xDEADBEEF"
>>> txOut' .^ txOutAddressL
"0xDEADBEEF"
```

### Cardano Transaction Types

In [8]:
{-# LANGUAGE DerivingStrategies #-}
{-# LANGUAGE DerivingVia #-}

import Data.ByteString
import Data.Semigroup

newtype Coin = Coin Integer
  deriving (Show, Eq)
  deriving (Semigroup, Monoid) via Sum Integer

type Address = KeyHash 'Payment

type TxId = Hash Blake2b256

type PublicKey = ByteString
type Signature = ByteString
type Witnesses = [(PublicKey, Signature)]

```haskell
data Tx = Tx
  { txBody :: TxBody
  , txWitnesses :: Witnesses
  }
```

```haskell
data TxBody = TxBody
  { txBodyInputs :: Set TxIn
  , txBodyOutputs :: [TxOut]
  }
```

```haskell
data TxOut = TxOut
  { txOutAddress :: Address
  , txOutValue :: Coin
  }
```

In [9]:
data TxIn = TxIn
  { txInId :: TxId
  , txInIx :: Int
  }

```haskell
type UTxO = Map TxIn TxOut
```

```haskell
isTxBodyBalanced :: TxBody -> (TxIn -> TxOut) -> Bool
isTxBodyBalanced TxBody{txBodyInputs, txBodyOutputs} txOutLookup =
  let consumed = foldMap (txOutValue . txOutLookup) txBodyInputs
      produced = foldMap txOutValue txBodyOutputs
   in consumed == produced
```

## Cardano Eras

* Preserve types and rules for every era forever
* Ability to extend types in a reusable manner
* Reuse parts of validation that have not changed from one era to another

Byron -> Shelley -> Allegra -> Mary -> Alonzo -> Babbage -> Conway

### Byron

```haskell
data Tx = Tx
  { txBody :: TxBody
  , txWitnesses :: Witnesses
  }
data TxBody = TxBody
  { txBodyInputs :: Set TxIn
  , txBodyOutputs :: [TxOut]
  }
data TxOut = TxOut
  { txOutAddress :: Address
  , txOutValue :: Coin
  }

isTxBodyBalanced :: TxBody -> (TxIn -> TxOut) -> Bool
isTxBodyBalanced TxBody{txBodyInputs, txBodyOutputs} txOutLookup =
  let consumed = foldMap (txOutValue . txOutLookup) txBodyInputs
      produced = foldMap txOutValue txBodyOutputs
   in consumed == produced
```

## Shelley design process

```haskell
data ShelleyTxOut = ShelleyTxOut
  { txOutAddress :: Address
  , txOutValue :: Coin
  }
```

```haskell
class EraTxOut era where
  type TxOut era :: Type
  addressTxOutL :: Lens' (TxOut era) Address
  valueTxOutL :: Lens' (TxOut era) Coin
```

In [12]:
{-# LANGUAGE TypeFamilies #-}
import Data.Kind (Type)

class EraTxOut era where
  type TxOut era :: Type
  addressTxOutL :: Lens' (TxOut era) Address

: 

```haskell
{-# LANGUAGE TypeFamilyDependencies #-}
import Data.Kind (Type)

class EraTxOut era where
  type TxOut era = (r :: Type) | r -> era
  addressTxOutL :: Lens' (TxOut era) Address
  valueTxOutL :: Lens' (TxOut era) Coin
```

In [13]:
{-# LANGUAGE TypeFamilyDependencies #-}
import Data.Kind (Type)

class EraTxOut era where
  type TxOut era = (r :: Type) | r -> era
  addressTxOutL :: Lens' (TxOut era) Address
  valueTxOutL :: Lens' (TxOut era) Coin

data ShelleyTxOut = ShelleyTxOut
  { txOutAddress :: Address
  , txOutValue :: Coin
  }

data Shelley
instance EraTxOut Shelley where
  type TxOut Shelley = ShelleyTxOut

data Allegra
instance EraTxOut Allegra where
  type TxOut Allegra = ShelleyTxOut

: 

### Another use case for phantom type

It can help us solve the issue with injective type families 

```haskell
data ShelleyTxOut era = ShelleyTxOut
  { txOutAddress :: Address
  , txOutValue :: Coin
  }
  deriving (Eq, Show)

instance EraTxOut Shelley where
  type TxOut Shelley = ShelleyTxOut Shelley

instance EraTxOut Allegra where
  type TxOut Allegra = ShelleyTxOut Allegra
```

## Full Solution

In [14]:
class (Eq (Value era), Monoid (Value era)) => EraTxOut era where
  type TxOut era = (r :: Type) | r -> era
  type Value era :: Type
  addressTxOutL :: Lens' (TxOut era) Address
  valueTxOutL :: Lens' (TxOut era) (Value era)

In [15]:
import Data.Set (Set)

class EraTxOut era => EraTxBody era where
  type TxBody era = (r :: Type) | r -> era
  inputsTxBodyL :: Lens' (TxBody era) (Set TxIn)
  outputsTxBodyL :: Lens' (TxBody era) [TxOut era]

In [16]:
class EraTxBody era => EraTx era where
  type Tx era = (r :: Type) | r -> era
  bodyTxL :: Lens' (Tx era) (TxBody era)
  witnessesTxL :: Lens' (Tx era) Witnesses

In [17]:
import Lens.Micro.Extras (view)

txBodyBalance
  :: EraTxBody era
  => TxBody era
  -> (TxIn -> TxOut era)
  -> (Value era, Value era)
txBodyBalance body txOutLookup = (consumed, produced)
  where
    consumed = 
      foldMap (view valueTxOutL . txOutLookup) (body ^. inputsTxBodyL)
    produced = 
      foldMap (view valueTxOutL) (body ^. outputsTxBodyL)

## Shelley

In [26]:
data ShelleyTxOut era = ShelleyTxOut
  { txOutAddress :: Address
  , txOutValue :: Value era
  }

data ShelleyTxBody era = ShelleyTxBody
  { txBodyInputs :: Set TxIn
  , txBodyOutputs :: [TxOut era]
  }

data ShelleyTx era = ShelleyTx
  { txBody :: TxBody era
  , txWitnesses :: Witnesses
  }

In [19]:
data Shelley

instance EraTxOut Shelley where
  type TxOut Shelley = ShelleyTxOut Shelley
  type Value Shelley = Coin
  addressTxOutL =
    lens txOutAddress $ \out addr -> out{txOutAddress = addr}
  valueTxOutL =
    lens txOutValue $ \out val -> out{txOutValue = val}

instance EraTxBody Shelley where
  type TxBody Shelley = ShelleyTxBody Shelley
  inputsTxBodyL =
    lens txBodyInputs $ \body ins -> body{txBodyInputs = ins}
  outputsTxBodyL =
    lens txBodyOutputs $ \body outs -> body{txBodyOutputs = outs}

instance EraTx Shelley where
  type Tx Shelley = ShelleyTx Shelley
  bodyTxL =
    lens txBody $ \tx body -> tx{txBody = body}
  witnessesTxL =
    lens txWitnesses $ \tx outs -> tx{txWitnesses = outs}

In [ ]:
isShelleyTxBodyBalanced
  :: EraTxBody era
  => TxBody era
  -> (TxIn -> TxOut era)
  -> Bool
isShelleyTxBodyBalanced body txOutLookup =
  uncurry (==) $ txBodyBalance body txOutLookup

```haskell
txBodyBalance
  :: EraTxBody era
  => TxBody era
  -> (TxIn -> TxOut era)
  -> (Value era, Value era)
txBodyBalance body txOutLookup = (consumed, produced)
  where
    consumed = 
      foldMap (view valueTxOutL . txOutLookup) (body ^. inputsTxBodyL)
    produced = 
      foldMap (view valueTxOutL) (body ^. outputsTxBodyL)
```

## Mary

In [ ]:
newtype MultiAsset = MultiAsset (Map.Map (KeyHash 'Minting) Integer)
  deriving (Show, Eq)

instance Semigroup MultiAsset where
  MultiAsset m1 <> MultiAsset m2 = MultiAsset $ Map.unionWith (+) m1 m2

instance Monoid MultiAsset where
  mempty = MultiAsset mempty

data MaryValue = MaryValue Coin MultiAsset
  deriving (Show, Eq)

instance Semigroup MaryValue where
  MaryValue c1 m1 <> MaryValue c2 m2 = MaryValue (c1 <> c2) (m1 <> m2)

instance Monoid MaryValue where
  mempty = MaryValue mempty mempty

In [29]:
data Mary

instance EraTxOut Mary where
  type TxOut Mary = ShelleyTxOut Mary
  type Value Mary = MaryValue
  addressTxOutL = 
    lens txOutAddress $ \out addr -> out{txOutAddress = addr}
  valueTxOutL = 
    lens txOutValue $ \out val -> out{txOutValue = val}

```haskell
data ShelleyTxOut era = ShelleyTxOut
  { txOutAddress :: Address
  , txOutValue :: Value era
  }
```

In [30]:
data MaryTxBody era = MaryTxBody
  { txBodyInputs :: Set TxIn
  , txBodyOutputs :: [TxOut era]
  , txBodyMint :: MultiAsset
  }

class EraTxBody era => MaryEraTxBody era where
  mintTxBodyL :: Lens' (TxBody era) MultiAsset

In [31]:
instance EraTxBody Mary where
  type TxBody Mary = MaryTxBody Mary
  inputsTxBodyL = 
    lens txBodyInputs $ \body ins -> body{txBodyInputs = ins}
  outputsTxBodyL = 
    lens txBodyOutputs $ \body outs -> body{txBodyOutputs = outs}

instance MaryEraTxBody Mary where
  mintTxBodyL = 
    lens txBodyMint $ \body mint -> body{txBodyMint = mint}

In [32]:
isMaryTxBodyBalanced
  :: (MaryEraTxBody era, Value era ~ MaryValue)
  => TxBody era
  -> (TxIn -> TxOut era)
  -> Bool
isMaryTxBodyBalanced body txOutLookup =
  consumed == produced <> MaryValue (Coin 0) (body ^. mintTxBodyL)
  where
    (consumed, produced) = txBodyBalance body txOutLookup

```haskell
txBodyBalance
  :: EraTxBody era
  => TxBody era
  -> (TxIn -> TxOut era)
  -> (Value era, Value era)
txBodyBalance body txOutLookup = (consumed, produced)
  where
    consumed = 
      foldMap (view valueTxOutL . txOutLookup) (body ^. inputsTxBodyL)
    produced = 
      foldMap (view valueTxOutL) (body ^. outputsTxBodyL)
```

## Questions?

```haskell
class (Eq (Value era), Monoid (Value era)) => EraTxOut era where
  type TxOut era = (r :: Type) | r -> era
  type Value era :: Type
  addressTxOutL :: Lens' (TxOut era) Address
  valueTxOutL :: Lens' (TxOut era) (Value era)

class EraTxOut era => EraTxBody era where
  type TxBody era = (r :: Type) | r -> era
  inputsTxBodyL :: Lens' (TxBody era) (Set TxIn)
  outputsTxBodyL :: Lens' (TxBody era) [TxOut era]

class EraTxBody era => MaryEraTxBody era where
  mintTxBodyL :: Lens' (TxBody era) MultiAsset

class EraTxBody era => EraTx era where
  type Tx era = (r :: Type) | r -> era
  bodyTxL :: Lens' (Tx era) (TxBody era)
  witnessesTxL :: Lens' (Tx era) Witnesses
```